In [46]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
api_key = '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'
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",  
  api_key=api_key,           
)

In [21]:
completion = client.chat.completions.create(
  model="deepseek/deepseek-chat-v3.1:free", 
  messages=[
    {"role": "user", "content": "answer in one word, what is the capital of morocco"}
  ]
)

In [22]:
print(completion.choices[0].message.content)

Rabat


In [25]:
with open("amine_website_data.txt", "r" , encoding="utf-8") as f:
    raw_text = f.read()
print(raw_text[:100])    

[Website Info]
Hey, I’m Amine from Morocco, and I created this website to detect Moroccan money, inc


In [29]:
text_splitter = CharacterTextSplitter(
    separator="\n\n", 
    chunk_size= 500, 
    chunk_overlap = 50, 
)
texts = text_splitter.split_text(raw_text)
lens = []
for chunk in range(len(texts)):
    lens.append(len(texts[chunk]))
print(lens)    

[423, 278, 431, 409, 425, 464, 473, 460, 216, 496, 420, 421, 447, 392, 380, 145]


In [30]:
embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(texts, embeddings)

In [47]:
llm = ChatOpenAI(
    model_name="deepseek/deepseek-chat-v3.1:free",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=api_key,
    temperature=0.5
)

In [48]:
retriever = vectorstore.as_retriever()
rag_chain = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = 'stuff', 
    retriever=retriever,
)

In [50]:
print(rag_chain.run("amine's favorite series ?"))

Based on the provided context, Amine's favorite series include:

- Stranger Things (all-time favorite)
- Game of Thrones
- Alice in the Borderland
- Teen Wolf


In [51]:
print(rag_chain.run("give me amine's instagram"))

Based on the context provided, Amine's Instagram handle is **@aminecanflyy**.


In [53]:
print(rag_chain.run("give me some amine's favorite anime character"))

Based on the context provided, Amine's favorite anime characters are:

- Eren Jaeger (Attack on Titan)
- Tanjiro Kamado (Demon Slayer)
- Mikasa (Attack on Titan)
- Erwin Smith (Attack on Titan)
- Goku (Dragon Ball)
- Android 18 (Dragon Ball)
- Muichiro Tokito (Demon Slayer)

These characters come from some of his favorite anime series, including *Attack on Titan*, *Demon Slayer*, and *Dragon Ball*.
